In [1]:
import pymongo
import json
import csv

import sys
!{sys.executable} -m pip install tqdm
from tqdm import tqdm

from pymongo import MongoClient

In [2]:
# To establish connection:
db = MongoClient('localhost', 27017)['EDDB']


In [3]:
# # import all of the files

# files = ['commodities', 'factions', 'modules', 'stations', 'systems_populated']

# for x in files:
#     print(x)
#     with open(f"{x}.json") as f:
#         db[x].insert_many(json.load(f))

In [4]:
# Function to convert a CSV to JSON
def make_json(data, csvFilePath): 
      
    # Open a csv reader called DictReader 
    with open(csvFilePath, encoding='utf-8') as csvf: 
        csvReader = csv.DictReader(csvf) 
          
        # Convert each row into a dictionary  
        # and add it to data 
        for rows in tqdm(csvReader, desc="Reading file"): 
              
            # Assuming a column named 'No' to 
            # be the primary key 
            key = rows['id'] 
            data[key] = rows

In [8]:
# import listing data that is in csv file
station_data = []

with open("stations.json") as f:
        station_data = json.load(f)

station_ids = set()

system_data = []

with open("systems_populated.json") as f:
        system_data = json.load(f)

system_ids = set()
    
for dic in tqdm(system_data, desc="Building system id set"):
    system_ids.add(dic['id'])
    
for dic in tqdm(station_data, desc="Building station id set"):
    if(dic['system_id'] in system_ids):
        station_ids.add(dic['id'])
    
chunk_size = 1000

data_dict = {}
make_json(data_dict, 'listings.csv')


data_to_insert = []
for key in tqdm(data_dict, desc="Converting data to correct list of JSON..."):
    data_to_insert.append(data_dict[key])

for dic in tqdm(data_to_insert, desc="Converting strings to ints..."):
    for key in dic:
        if(dic[key] == ''):
            dic[key] = 0
        else:
            dic[key] = int(dic[key])

filtered_data = []

for dic in tqdm(data_to_insert, desc="Filtering out listings from systems we don't have"):
    if(dic['station_id'] in station_ids):
        filtered_data.append(dic)
            
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in tqdm(range(0, len(lst), n), desc="Chunking..."):
        yield lst[i:i + n]

chunks = list(chunks(filtered_data, chunk_size))
        
for chunk in tqdm(chunks, desc="Uploading Chunks"):
    db['listings'].insert_many(chunk)

Building station id set: 100%|██████████| 66024/66024 [00:00<00:00, 1100379.19it/s]
Reading file: 3931637it [00:13, 293077.69it/s]
Converting strings to ints...: 100%|██████████| 3931637/3931637 [00:16<00:00, 234640.58it/s]
Filtering out listings from systems we don't have: 100%|██████████| 3931637/3931637 [00:02<00:00, 1954096.17it/s]
Uploading Chunks: 100%|██████████| 3932/3932 [01:24<00:00, 46.79it/s]
